In [1]:
import pandas as pd
import sys
import requests
from io import StringIO

sys.path.append("../src")
from data_fetching.sirene import get_company_info

# Idée générale

Il s'agit dans ce notebook de mettre en place l'idée de cet [issue](https://github.com/dataforgoodfr/offseason_missiontransition_simulateur/issues/8). 

Formellement, entraîner un modèle qui à un couple `(aid, company)` est en mesure de rendre un `score` entre 0 et 1 representant les chances de succès de la demande d'aide pour savoir à quel point il est pertinent de proposer précisément cette `aid` à cette `company`.

# Préparation du dataset

## Données ADEME
Nous disposons d'un ensemble d'exemples de $n$ couples $\mathcal{S} = \{(a_i, c_i)\} _{i\leq n}$ qui ont fait l'objet d'une attribution de l'ADEME : pour tout $i\leq n$, l'entreprise $c_i$ a reçu l'aide $a_i$ proposée par l'ADEME.

Nous voulons créer un dataset labelisé, $\mathcal{D} = \{[(a_i, c_i), y_i]\}_{i\leq N}$ où $y_i$ représente les chances que l'entreprise $c_i$ obtienne (ou ait intérêt à obtenir) l'aide $a_i$.

$\mathcal{S}$ nous fournit un premier dataset avec des labels $y_i=1$, puisqu'il s'agit d'exemples où l'entreprise a effectivement obtenu l'aide en question.

Pour avoir un dataset complet, il faudrait avoir des exemples de couples $(a_i, c_i)$, avec un score $y_i$ inférieur à 1. 

## Augmentation du dataset

L'idée est donc la suivante : pour chaque aide $a_i$ dans un couple dans $\mathcal{S}$, on sait que seule l'entreprise $c_i$ a obtenu cette aide, autrement dit tous les couples $(a_i, c_j)$ pour $j\neq i$ peuvent être vu comme des exemples avec un label $y=0$.

Ceci nous fournit donc $n(n-1)$ exemples avec un label $y=0$ ($n-1$ exemples pour les $n$ aides). Pour éviter d'avoir un dataset trop déséquilibré, on peut en garder $n$ au hasard afin de former un dataset de taille $2n$.

In [2]:
### Dataset preparation

# Modélisation

## Features

### Côté Mission-Transition

In [3]:
mt_aids = pd.read_csv("https://dataforgood.slite.com/api/files/0wYKC93pC/data-1634140125310.csv")

In [4]:
mt_aids.head()

,id,aid_advisor_id,funder_id,ulid,source_id,name,perimeter,goal,beneficiary,aid_details,...,aid_id-2,aid_type_id,id-3,name-3,slug-2,aid_id-3,environmental_topic_id,id-4,name-4,slug-3
0,198,NaN,10.0,017c7a50-c5ea-5d3b-e8bc-569be89a32fd,at_94683,Financer les études et tests préalables aux in...,NATIONAL,NaN,NaN,<p>\n <strong>\n Aide - « Études et tests pré...,...,198.0,1.0,NaN,NaN,NaN,198.0,1.0,NaN,NaN,NaN
1,187,NaN,10.0,017c7a50-c4c6-704e-7235-0b41ca6f711e,at_58447,Soutenir financièrement le développement de l'...,NATIONAL,NaN,NaN,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,...,187.0,1.0,NaN,NaN,NaN,187.0,1.0,NaN,NaN,NaN
2,13,NaN,3.0,017c7a50-b75f-c83d-01bc-49b7055c407f,at_90955,Soutenir financièrement l'intégration de matiè...,REGIONAL,NaN,NaN,<p>\n <p>\n <p>\n <p>\n <p>\n ORPLAS...,...,13.0,1.0,NaN,NaN,NaN,13.0,1.0,13.0,Stockage de l'énergie,stockage-de-lenergie
3,13,NaN,3.0,017c7a50-b75f-c83d-01bc-49b7055c407f,at_90955,Soutenir financièrement l'intégration de matiè...,REGIONAL,NaN,NaN,<p>\n <p>\n <p>\n <p>\n <p>\n ORPLAS...,...,13.0,1.0,NaN,NaN,NaN,13.0,2.0,13.0,Stockage de l'énergie,stockage-de-lenergie
4,213,NaN,10.0,017c7a50-c7a5-fc2f-8bb3-ca4b562a0715,at_94687,Financer les études de préfiguration d'une dém...,NATIONAL,NaN,NaN,<p>\n Aide - « Étude de préfiguration d'une dé...,...,213.0,1.0,NaN,NaN,NaN,213.0,3.0,NaN,NaN,NaN


In [5]:
mt_aids.sample(1).iloc[0]

id                                                                           218
aid_advisor_id                                                               NaN
funder_id                                                                   61.0
ulid                                        017c7a50-c931-4626-a3d5-96b3f00b9db8
source_id                                                               at_30859
name                             Organiser un colloque scientifique et technique
perimeter                                                               REGIONAL
goal                                                                         NaN
beneficiary                                                                  NaN
aid_details                    <p>\n <strong>\n  COLLOQUES SCIENTIFIQUES ET T...
eligibility                                                                  NaN
conditions                                                                   NaN
funding_source_url          

Natural language features :
- name
- aid_details
- eligibility

Numeric features :
- subvention_rate_upper_bound
- subvention_rate_lower_bound

Categorical features :
- funding_types

### Côté ADEME

Côté ADEME, on ne trouvera pas plus de détail sur les aides, en revanche on obtient le nom de l'entreprise, ce qui nous permet d'obtenir les informations récupérables par l'API SIRENE comme  [ici](https://github.com/dataforgoodfr/offseason_missiontransition_simulateur/issues/4).

In [6]:
resp = requests.get("https://koumoul.com/data-fair/api/v1/datasets/les-aides-financieres-de-l'ademe/full")
ademe_df = pd.read_csv(StringIO(resp.content.decode()))

In [7]:
ademe_df.head()

,Nom de l attribuant,idAttribuant,dateConvention,referenceDecision,nomBeneficiaire,idBeneficiaire,objet,montant,nature,conditionsVersement,...,notificationUE,pourcentageSubvention,location.lat,location.lon,DEPET,naf1etlib,naf2etlib,naf3etlib,naf4etlib,naf5etlib
0,ADEME,38529030900454,2021-04-12,21BFD0081,CLEMENT SAS,7.269205e+13,TREMPLIN pour la transition écologique des PME,6500.0,aide en numéraire,Echelonné,...,NaN,1.0,46.671862,4.365152,71,Industrie manufacturière,Fabrication de machines et équipements n.c.a.,Fabrication de machines de formage des métaux ...,Fabrication de machines de formage des métaux,Fabrication de machines-outils pour le travail...
1,ADEME,38529030900454,2021-03-25,21BFD0119,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,3.950618e+13,TREMPLIN pour la transition écologique des PME,12150.0,aide en numéraire,Echelonné,...,NaN,1.0,47.105037,5.504615,39,Industrie manufacturière,"Fabrication de produits métalliques, à l'excep...",Traitement et revêtement des métaux ; usinage,Usinage,Mécanique industrielle
2,ADEME,38529030900454,2021-03-23,21BFD0123,TRANSDEV BFC SUD,3.391339e+13,TREMPLIN pour la transition écologique des PME,16000.0,aide en numéraire,Echelonné,...,NaN,1.0,46.790418,4.833945,71,Transports et entreposage,Transports terrestres et transport par conduites,Autres transports terrestres de voyageurs,Autres transports terrestres de voyageurs n.c.a.,Transports routiers réguliers de voyageurs
3,ADEME,38529030900454,2021-03-23,21BFD0137,LOCA TRAVAUX EURL,4.999836e+13,TREMPLIN pour la transition écologique des PME,16400.0,aide en numéraire,Echelonné,...,NaN,1.0,47.623341,6.957088,90,Activités de services administratifs et de sou...,Activités de location et location-bail,"Location et location-bail d'autres machines, é...",Location et location-bail de machines et équip...,Location et location-bail de machines et équip...
4,ADEME,38529030900454,2021-03-26,21BFD0142,RODESCHINI SAS,3.008465e+13,TREMPLIN pour la transition écologique des PME,21500.0,aide en numéraire,Echelonné,...,NaN,1.0,NaN,NaN,70,Construction,Travaux de construction spécialisés,Autres travaux de construction spécialisés,Autres travaux de construction spécialisés n.c.a.,Travaux de maçonnerie générale et gros œuvre d...


In [8]:
ademe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12696 entries, 0 to 12695
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Nom de l attribuant    12696 non-null  object 
 1   idAttribuant           12696 non-null  int64  
 2   dateConvention         12696 non-null  object 
 3   referenceDecision      12696 non-null  object 
 4   nomBeneficiaire        12696 non-null  object 
 5   idBeneficiaire         12463 non-null  float64
 6   objet                  12696 non-null  object 
 7   montant                12696 non-null  float64
 8   nature                 12696 non-null  object 
 9   conditionsVersement    12696 non-null  object 
 10  datesPeriodeVersement  12696 non-null  object 
 11  idRAE                  0 non-null      float64
 12  notificationUE         695 non-null    float64
 13  pourcentageSubvention  12696 non-null  float64
 14  location.lat           11676 non-null  float64
 15  lo

In [9]:
campanies = ademe_df.nomBeneficiaire.unique()
len(campanies)

7743

On a donc 7743 entreprises

In [10]:
get_company_info(campanies[0])

{'siren': '726920531',
 'creation_date': '1969-01-01',
 'workforce': {'code': '11', 'date': '2019', 'value': [10, 19]},
 'category': {'code': 'PME', 'date': '2019'},
 'activity': {'code': '25.62B', 'date': 'NAFRev2'},
 'ess': 'N'}

En réalité le libellé NAF est déjà donné dans les aides ADEME, en revanche on pourra utiliser les informations liées à la taille de l'entreprise.

## Modèle

Les features en langage naturel peuvent être traités avec [SBERT](https://www.sbert.net/docs/quickstart.html) pour ensuite entraîner un modèle de classification.